# Chapter-13 Loading and Preprocessing Data with Tensorflow

In [1]:
import tensorflow as tf

## The Data API

The whole Data API revolves around the concept of a dataset, this represents a sequence of data items. Usually datasets will be gradually read from the dist, but for simplicity let's create a dataset entirely in RAM using tf.data.Dataset.from_tensor_slices():

In [2]:
X = tf.range(10) # any data tensor
dataset = tf.data.Dataset.from_tensor_slices(X)

In [3]:
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

The from_tensor_slices() function takes a tensor and creates a tf.data.Dataset
whose elements are all the slices of X (along the first dimension), so this dataset con‐
tains 10 items: tensors 0, 1, 2, ..., 9. In this case we would have obtained the same
dataset if we had used tf.data.Dataset.range(10)

In [4]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


## Chaining Transformations

In [5]:
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


You can also transform the items by calling the map() method. For example, this cre‐
ates a new dataset with all items doubled:

In [7]:
dataset = dataset.map(lambda x: x * 2) # Items: [0, 2, 4, 6, 8, 10, 12]

This function is the one you will call to apply any preprocessing you want to your
data. Sometimes this will include computations that can be quite intensive, such as
reshaping or rotating an image, so you will usually want to spawn multiple threads to
speed things up: it’s as simple as setting the num_parallel_calls argument. Note that
the function you pass to the map() method must be convertible to a TF Function

While the map() method applies a transformation to each item, the apply() method
applies a transformation to the dataset as a whole. For example, the following code
applies the unbatch() function to the dataset (this function is currently experimental,
but it will most likely move to the core API in a future release). Each item in the new
dataset will be a single-integer tensor instead of a batch of seven integers:

In [9]:
dataset = dataset.apply(tf.data.experimental.unbatch())

Instructions for updating:
Use `tf.data.Dataset.unbatch()`.


In [11]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(14, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(18, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(14, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(18, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(14, sh

It is also possible to simply filter the dataset using the filter() method:

In [12]:
dataset = dataset.filter(lambda x: x < 10)

In [13]:
for item in dataset.take(3):
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)


## Shuffling the Data

It will create a new dataset
that will start by filling up a buffer with the first items of the source dataset. Then,
whenever it is asked for an item, it will pull one out randomly from the buffer and
replace it with a fresh one from the source dataset, until it has iterated entirely
through the source dataset. At this point it continues to pull out items randomly from
the buffer until it is empty. You must specify the buffer size, and it is important to
make it large enough, or else shuffling will not be very effective.

In [15]:
dataset = tf.data.Dataset.range(10).repeat(3)
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int64)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int64)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int64)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int64)
tf.Tensor([3 6], shape=(2,), dtype=int64)


For a large dataset that does not fit in memory, this simple shuffling-buffer approach
may not be sufficient, since the buffer will be small compared to the dataset. One sol‐
ution is to shuffle the source data itself (for example, on Linux you can shuffle text
files using the shuf command). This will definitely improve shuffling a lot! Even if
the source data is shuffled, you will usually want to shuffle it some more, or else the
same order will be repeated at each epoch, and the model may end up being biased
(e.g., due to some spurious patterns present by chance in the source data’s order). To
shuffle the instances some more, a common approach is to split the source data into
multiple files, then read them in a random order during training. However, instances
located in the same file will still end up close to each other. To avoid this you can pick
multiple files randomly and read them simultaneously, interleaving their records.
Then on top of that you can add a shuffling buffer using the shuffle() method. If all the next five file paths from the filepath_dataset and interleave them the same way,
and so on until it runs out of file paths.

By default, interleave() does not use parallelism; it just reads one line at a time
from each file, sequentially. If you want it to actually read files in parallel, you can set
the num_parallel_calls argument to the number of threads you want (note that the
map() method also has this argument). You can even set it to tf.data.experimen
tal.AUTOTUNE to make TensorFlow choose the right number of threads dynamically
based on the available CPU (however, this is an experimental feature for now). Let’s
look at what the dataset contains now:

## Preprocessing the Data